To go from fabry-perot scale to MHz use:
- freq_MHz =  fabry_perot_value * unit_blue (or unit_IR)

In [8]:
unit_blue = 780/421
unit_IR = 780/1013

blue_au = 488 # blue fabry-perot arbitrary unit
infrared_fp_au = 889.5 # infrared fabry-perot arbitrary unit
depumping_RF_Mhz = 82 # AOM+1 radio-frequency


blue_movement = +1
new_blue = blue_au + blue_movement
infrared_movement = - blue_movement*unit_blue/unit_IR
new_infrared = infrared_fp_au + infrared_movement
new_depumping = depumping_RF_Mhz + infrared_movement

print("new IR : ", new_infrared, "\nnew dep : ", new_depumping, "\nnew lue : ", new_blue)

new IR :  887.0938242280286 
new dep :  79.59382422802851 
new lue :  489


In [10]:
# to excite with depumping beam
794.9+80/unit_IR

898.7974358974359

In [3]:
889.2 + 160/unit_IR


1096.9948717948719

In [4]:
### for the blue
F3 = 295
F2 = F3+87.05/unit_blue
F1 = F2+51.44/unit_blue
F0 = F1+23.74/unit_blue

print(f" F0 = {F0}\n", f"F1 = {F1}\n", f"F2 = {F2}\n", f"F3 = {F3}")

 F0 = 382.5626025641026
 F1 = 369.74908974358976
 F2 = 341.9846794871795
 F3 = 295


In [5]:
### for the IR
F3 = 290.2            # write here the fabry-perot resonance for the 6p F=3
F2 = F3-87.05/unit_IR
F1 = F2-51.44/unit_IR
F0 = F1-23.74/unit_IR

print(f" F0 = {F0}\n", f"F1 = {F1}\n", f"F2 = {F2}\n", f"F3 = {F3}")

 F0 = 79.50898717948716
 F1 = 110.34055128205127
 F2 = 177.14660256410255
 F3 = 290.2


In [6]:
a_aus = 2.512 # intercept fitted
b = 1.1584 # angular coefficient fitted
da = 0.0734
db = 0.126
time_fit = 1

rate = b/a_aus*1000
err = ((da/a_aus)**2+(db/b)**2)**(1/2)*(b/a_aus)*1000
print("taking as denominator the N_ions in the start: ", rate, "+-", err)

a = (a_aus + (a_aus-b*time_fit/2))/2

rate = b/a*1000
err = ((da/a)**2+(db/b)**2)**(1/2)*(b/a)*1000
print("taking as denominator the N_ions in the half: ", rate, "+-", err)


taking as denominator the N_ions in the start:  461.14649681528664 +- 51.93759056668979
taking as denominator the N_ions in the half:  521.2383009359252 +- 59.25146554456595
